# Qualifying Analysis  

In [1]:

## -------Import required libraries-------
import fastf1
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.font_manager import FontProperties
import pandas as pd
import numpy as np
import hvplot.pandas
import holoviews as hv
from holoviews import opts
import panel as pn
import warnings
warnings.filterwarnings('ignore')

hv.extension('bokeh')
pn.extension()

In [3]:
## -------Load the data-------
fastf1.Cache.enable_cache(r"D:\PYTON PROGRAMMING\PYTHON FILES\Data-Visualization-Using-Python\STREAMLIT & PANEL\F1 RACE ANALYSIS PROJECTS\F1 Candian GP 2025\cache")
quali = fastf1.get_session(2025, 'canada', 'Q')
quali.load()

## -------Load the Font-------

core           INFO 	Loading data for Canadian Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['63', '1', '81', '12', '44', '14', '4', '16', '6', '23', '22', '43', '27', '87', '31', '5', '55', '18', '30', '10']


In [4]:
driver_colors = {
    'VER': '#1E41FF',  # Red Bull
    'TSU': '#3671C6',  # Red Bull
    'ANT': '#00D2BE',  # Mercedes
    'RUS': '#00D2BE',  # Mercedes
    'LEC': '#DC0000',  # Ferrari
    'HAM': '#DC0000',  # Ferrari
    'NOR': '#FF8700',  # McLaren
    'PIA': '#FF8700',  # McLaren
    'ALO': '#229971',  # Aston Martin
    'STR': '#229971',  # Aston Martin
    'ALB': '#005AFF',  # Williams
    'SAI': '#005AFF',  # Williams
    'COL': '#FF87BC',  # Alpine
    'GAS': '#FF87BC',  # Alpine
    'BER': "#ACAAAA",  # Sauber
    'OCO': "#969393",  # Sauber
    'LAW': '#2B4562',  # RB
    'HAD': '#2B4562',  # RB
    'BOR': "#21FB05",  # Haas
    'HUL': "#34F704"   # Haas
}


### Telemetry Analysis 

In [13]:
# Function to extract telemetry
def get_driver_telemetry(driver, metric):
    lap = quali.laps.pick_driver(driver).pick_fastest()
    tel = lap.get_telemetry().add_distance()
    df = tel[['Distance', metric]].copy()
    df['Driver'] = driver.upper()
    return df

# Combine two drivers' telemetry
def get_comparison_telemetry(driver1, driver2, metric):
    df1 = get_driver_telemetry(driver1, metric)
    df2 = get_driver_telemetry(driver2, metric)
    combined = pd.concat([df1, df2], ignore_index=True)
    return combined

# Plotting with custom color per driver
def plot_driver_comparison(driver1, driver2, metric):
    df = get_comparison_telemetry(driver1, driver2, metric)

    # Create individual plots per driver
    curves = []
    for driver in [driver1.upper(), driver2.upper()]:
        driver_df = df[df['Driver'] == driver]
        curve = driver_df.hvplot.line(
            x = 'Distance',
            y =  metric,
            label = driver,
            line_width = 4,
            width = 800,
            height = 400,
            color = driver_colors.get(driver, 'gray')  # default to gray if color not found
        ).opts(show_grid=True,
                xticks=20,  # increase number of x-axis grid lines
                yticks=20,   # increase number of y-axis grid lines
        ) 
        curves.append(curve)

    # Overlay both plots
    return hv.Overlay(curves).opts(
        opts.Overlay(
            legend_position = 'top_left',
            legend_opts = {'background_fill_alpha' : 0.5}
        )
    )
# Dropdown selections
drivers = sorted(quali.laps['Driver'].unique().tolist())
metrics = ['Speed', 'Throttle', 'Brake']

driver1_select = pn.widgets.Select(name='Driver 1', options=drivers, value=drivers[0])
driver2_select = pn.widgets.Select(name='Driver 2', options=drivers, value=drivers[1])
metric_select = pn.widgets.Select(name='Metric', options=metrics, value='Speed')

# Binding interactive plot
interactive_plot = pn.bind(
    plot_driver_comparison,
    driver1=driver1_select,
    driver2=driver2_select,
    metric=metric_select
)

# Dashboard layout
dashboard = pn.Column(
    pn.pane.Markdown("## 📊 Driver Telemetry Comparison"),
    pn.Row(driver1_select, driver2_select, metric_select),
    interactive_plot
)

dashboard.servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'284a0331-b0c4-4c4b-8709-f8dc7466c697': {'version…

### Track Dominance 

In [ ]:
quali.laps.pick_driver('VER').get_telemetry().columns

Index(['Date', 'SessionTime', 'DriverAhead', 'DistanceToDriverAhead', 'Time',
       'RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'DRS', 'Source',
       'Distance', 'RelativeDistance', 'Status', 'X', 'Y', 'Z'],
      dtype='object')

In [ ]:
telemetry = quali.laps.get_telemetry().add_distance()

#

# Race Analysis

### Analysing Sector Times 

In [3]:
race = fastf1.get_session(2025, 'canada', 'R')
race.load()
print(race.laps.columns)
df = race.laps

core           INFO 	Loading data for Canadian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['63', '1', '12', '81', '16', '44', '14', '27', '31', '55', '87', '22', '43', '5', '10', '6', '18', '4', '30', '23']


Index(['Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint',
       'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
       'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
       'Compound', 'TyreLife', 'FreshTyre', 'Team', 'LapStartTime',
       'LapStartDate', 'TrackStatus', 'Position', 'Deleted', 'DeletedReason',
       'FastF1Generated', 'IsAccurate'],
      dtype='object')


In [17]:

import panel as pn

pn.extension(raw_css=[open("my_styles.css").read()])
drivers = df['Driver'].unique().tolist()
driver = pn.widgets.Select(name='Driver', options=drivers, value=drivers[0],   css_classes=['my-select'])  # 👈 This links your widget to 
driver

BokehModel(combine_events=True, render_bundle={'docs_json': {'01bab595-5fe2-4c62-9e95-798abd8b9aab': {'version…

In [21]:
import os

# Define folder structure
base_path = "f1-Canadian-GP-2025-Dashboard"
folders = [
    "data",
    "plots",
    "pages",
    "assets"
]

files = {
    "streamlit_app.py": "# Entry point for Streamlit app\n",
    "data/2025_canadian_gp.csv": "",  # You can leave this empty or write dummy content
    "plots/telemetry.py": "# Functions to create telemetry plots\n",
    "pages/Sector Times.py": "# Streamlit page: Sector Times analysis\n",
    "assets/styles.css": "/* Add custom CSS styles here */\n",
    "README.md": "# F1 Dashboard\n\nInteractive F1 race data analytics with Streamlit."
}

# Create base directory
os.makedirs(base_path, exist_ok=True)

# Create folders
for folder in folders:
    os.makedirs(os.path.join(base_path, folder), exist_ok=True)

# Create files with sample content
for filepath, content in files.items():
    full_path = os.path.join(base_path, filepath)
    os.makedirs(os.path.dirname(full_path), exist_ok=True)
    with open(full_path, "w") as f:
        f.write(content)

print(f"✅ Project structure created at: {os.path.abspath(base_path)}")


✅ Project structure created at: d:\PYTON PROGRAMMING\PYTHON FILES\Data-Visualization-Using-Python\PANEL\F1 RACE ANALYSIS PROJECTS\F1 Candian GP 2025\NOTEBOOK\f1-Canadian-GP-2025-Dashboard
